In [ ]:
import os
import io
import re
import numpy as np
import pandas as pd
import pysam
from tqdm.notebook import tqdm 

In [ ]:
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    res = pd.read_csv(
        io.StringIO(''.join(lines[:])),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})
    return res

foo = lambda x: pd.Series(x.split('CALLERS=')[1].split(';')[0])

# VCF file

In [ ]:
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    res = pd.read_csv(
        io.StringIO(''.join(lines[1:])),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})
    return res

foo = lambda x: pd.Series(x.split('CALLERS=')[1].split(';')[0])


vcf_pd = read_vcf("../data/2015-07-31_PATIENTID/PATIENTID-ensemble-annotated.vcf")


vcf_pd['callers'] = vcf_pd['INFO'].apply(foo)
vcf_pd['type'] = np.nan
vcf_pd['type'][vcf_pd['ALT'].str.len() - vcf_pd['REF'].str.len() == 0] = 'SNV'
vcf_pd['type'][vcf_pd['ALT'].str.len() - vcf_pd['REF'].str.len() > 0] = 'INS'
vcf_pd['type'][vcf_pd['ALT'].str.len() - vcf_pd['REF'].str.len() < 0] = 'DEL'
vcf_pd['type'][vcf_pd['ID'].str.contains('rs')] = 'SNP'
vcf_pd['freebayes'] = vcf_pd['INFO'].str.contains('freebayes')
vcf_pd['vardict'] = vcf_pd['INFO'].str.contains('vardict')
vcf_pd['varscan'] = vcf_pd['INFO'].str.contains('varscan')

vcf_pd[vcf_pd['type'] == 'INS']

In [ ]:
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    res = pd.read_csv(
        io.StringIO(''.join(lines[:])),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})
    return res

foo = lambda x: pd.Series(x.split('CALLERS=')[1].split(';')[0])

vcf_pd = read_vcf("../data/2015-07-31_healthy_CUHKCD_healthy-AB010/healthy_CUHKCD_healthy-AB010-ensemble-annotated.vcf")

vcf_pd['callers'] = vcf_pd['INFO'].apply(foo)
vcf_pd['type'] = np.nan
vcf_pd['type'][vcf_pd['ALT'].str.len() - vcf_pd['REF'].str.len() == 0] = 'SNV'
vcf_pd['type'][vcf_pd['ALT'].str.len() - vcf_pd['REF'].str.len() > 0] = 'INS'
vcf_pd['type'][vcf_pd['ALT'].str.len() - vcf_pd['REF'].str.len() < 0] = 'DEL'
vcf_pd['type'][vcf_pd['ID'].str.contains('rs')] = 'SNP'
vcf_pd['freebayes'] = vcf_pd['INFO'].str.contains('freebayes')
vcf_pd['vardict'] = vcf_pd['INFO'].str.contains('vardict')
vcf_pd['varscan'] = vcf_pd['INFO'].str.contains('varscan')

vcf_pd[vcf_pd['type'] == 'INS']

In [ ]:
vcf_pd.groupby('CHROM').size()

# Filter Bam w.r.t. VCF file

Important notes:
1. Indexing
    - SAM is a 1-index based file
    - VCF is a 1-index based file
    - pysam is a 0-index based tool

2. Paired-End sequencing
    - BAM/SAM are storing the resverse complementary of reversed reads as sequence

3. Mapping issues
    - some reads are not mapped -> no CIGAR string + no read.reference_end

In [ ]:
samfile = pysam.AlignmentFile("../data/healthy_chr22_merged-ready.bam", "rb")

# ititiate list of reads to remove
reads2remove = []
log_dict = {"position":[],"type":[],
            "total_reads":[], 'supporting_reads':[],
            "problematic_reads":[]}

# iterate over positions
for ci, mutation in tqdm(vcf_pd.iterrows(), total=vcf_pd.shape[0]):
    c = 0 # number of reads supporting the considered mutation
    t = 0 # total number of reads at that position
    p = 0 # number pf reads with issues
    for read in samfile.fetch(mutation['CHROM'], mutation['POS']-1, mutation['POS']): 
        t += 1
        seq = read.query_alignment_sequence
        pos = (mutation['POS'] -1) - read.reference_start + 1 
    if mutation['type'] == 'INS':
        cond1 = (seq[pos-1:  pos-1+len(mutation['ALT'])] == mutation['ALT'])
        if cond1: 
            cigar = read.cigarstring
            if cigar is None:
                p += 1
            else:
                cigar_pos = re.split('M|I|D|N|S|H|P|=|X',cigar)[:-1]
                cigar_states = re.split('[0-9]+',cigar)[1:]
                cond2 = False
                if 'I' in cigar_states:
                    cumul = 0
                    indel_pos = None
                    for i, cp in enumerate(cigar_pos):
                        if (cigar_states[i] != 'S') and (cumul <= pos):
                            cumul += -int(cp) if cp == 'D' else int(cp)
                            indel_pos = i
                    if cigar_states[indel_pos] == 'I':
                        cond2 = True
                if cond1 and cond2:
                    c += 1
                    reads2remove.append(read.query_name)

    elif (mutation['type'] == 'SNV') or (mutation['type'] == 'SNP'):
        cigar = read.cigarstring
        #print(pos, seq[pos-2:  pos+2], seq[pos-1], mutation['ALT'],  mutation['REF'])
        if cigar is None:
            p += 1
        else:
            cigar_pos = re.split('M|I|D|N|S|H|P|=|X',cigar)[:-1]
            cigar_states = re.split('[0-9]+',cigar)[1:]
            if 'D' in cigar_states:
                if 'S' in cigar_states:
                    Spos = [i for i, x in enumerate(cigar_states) if x == "S"] 
                    cigar_pos = [i for j, i in enumerate(cigar_pos) if j not in Spos]
                    cigar_states = [i for j, i in enumerate(cigar_states) if j not in Spos]
                Dpos = [i for i, x in enumerate(cigar_states) if x == "D"] 
                for dpos in Dpos:
                    if sum(list(map(int, cigar_pos[:dpos]))) < pos:
                        #print(sum(list(map(int, cigar_pos[:dpos]))), cigar)
                        pos = pos - int(cigar_pos[dpos])
                #print(len(seq), pos-1)
        if (seq[pos-1:pos-1+len(mutation['ALT'])] == mutation['ALT']):
            c += 1
            reads2remove.append(read.query_name)
    elif mutation['type'] == 'DEL':
        cond1 = (seq[pos-1:  pos-1+len(mutation['REF'])] != mutation['REF'])
        if cond1:
            cigar = read.cigarstring
            if cigar is None:
                p += 1
            else:
                cigar_pos = re.split('M|I|D|N|S|H|P|=|X',cigar)[:-1]
                cigar_states = re.split('[0-9]+',cigar)[1:]
                cond2 = False
                if 'D' in cigar_states:
                    cumul = 0
                    indel_pos = None
                    for i, cp in enumerate(cigar_pos):
                        if (cigar_pos[i] != 'S') and (cumul <= pos):
                            cumul += -int(cp) if cp == 'D' else int(cp)
                            indel_pos = i
                    if cigar_states[indel_pos] == 'D':
                        cond2 = True
                if cond1 and cond2:
                    c += 1
                    reads2remove.append(read.query_name)
    log_dict["position"].append(mutation['POS'])
    log_dict["type"].append(mutation['type'])
    log_dict["total_reads"].append(t)
    log_dict["supporting_reads"].append(c)
    log_dict["problematic_reads"].append(p)

samfile.close()

In [ ]:
log_pd = pd.DataFrame.from_dict(log_dict)
log_pd

In [ ]:
log_pd.shape[0]

In [ ]:
100*log_pd[(log_pd['supporting_reads'] == 0) & (log_pd['total_reads'] > 5)].shape[0]/log_pd[log_pd['total_reads'] > 5].shape[0]

In [ ]:
print(vcf_pd.shape)

In [ ]:
a = 'GCAGCCACTCAGGATGTTGGAACCTGGCCATCCCTGCTTCTTTCAGTGGGTGAGGTTGGTGGCTGCTCCACCTGTTCCAGGCACACCCTTAACAGAGGTGGCTGCTTGCTCTTTAAGCCAGCTTGGCCTTGCCTGGCATGCACAGGCCCCG'
b = 'GCAGCCACTCAGGATGTTGGAACCTGGCCATCCCTGCTTCTTTCAGTGGGTGAGGTTGGTGGCTGCTCCACCTGTTCCAGGCACACCCTTAACAGAGGTGGCTGCTTGCTCTTTAAGCCAGCTTGGCCTTGCCTGGCATGCACAGGCCCCG'

a == b